In [1]:

#@title Installs
!pip -q install --upgrade pip
!pip -q install torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip -q install diffusers transformers accelerate safetensors peft xformers datasets pillow



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.1 MB/s eta 0:00:00


In [2]:

#@title Environment Setup
import os, torch, json, time, random, pathlib
from PIL import Image
from typing import List
print("CUDA available:", torch.cuda.is_available())



CUDA available: True


In [3]:

#@title Model & pipeline bootstrap

from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline

# Choose a model id (SD 1.5 or SDXL). Change as needed.
MODEL_ID = "runwayml/stable-diffusion-v1-5"  # or "stabilityai/stable-diffusion-xl-base-1.0"
OUTPUT_DIR = "outputs_sd"
os.makedirs(OUTPUT_DIR, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained(MODEL_ID, torch_dtype=torch.float16 if device=="cuda" else torch.float32)
if device == "cuda":
    pipe.enable_attention_slicing()
    pipe.to(device)

def save_image(img: Image.Image, name: str) -> str:
    path = os.path.join(OUTPUT_DIR, name)
    img.save(path)
    return path



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [4]:

#@title Text-to-Image (generate ≥3 images)

prompts = [
    "A realistic photo of a chessboard mid-game with dramatic lighting",
    "An isometric chess diagram showing a knight fork, vector art style",
    "Cartoon illustration of a king and queen playing chess, flat style"
]
negative_prompt = "low quality, blurry, nsfw, text watermark, logo"


generated_paths = []
for i,p in enumerate(prompts):
    img = pipe(prompt=p, negative_prompt=negative_prompt, num_inference_steps=25, guidance_scale=7.5, height=512, width=512).images[0]
    path = save_image(img, f"txt2img_{i}.png")
    generated_paths.append(path)

generated_paths



  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

['outputs_sd/txt2img_0.png',
 'outputs_sd/txt2img_1.png',
 'outputs_sd/txt2img_2.png']

In [5]:

#@title Img2Img / Inpaint (perform one edit)
# For Img2Img: use the first generated image as init.
init_img_path = generated_paths[0]
init_image = Image.open(init_img_path).convert("RGB").resize((512,512))

img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32
).to(device)

edit_prompt = "Turn this chessboard into a neon cyberpunk hyper futuristic night scene"

edited = img2img(prompt=edit_prompt, image=init_image, strength=0.6, guidance_scale=8.0, num_inference_steps=30).images[0]
edited_path = save_image(edited, "img2img_edit.png")
edited_path


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

'outputs_sd/img2img_edit.png'

In [9]:
#@title Upload the LoRA training weights folder from other colab

from google.colab import files
files.upload();




Saving pytorch_lora_weights.safetensors to pytorch_lora_weights.safetensors


In [11]:

#@title Move Weights into a Folder

!mkdir lora_weights
!mv pytorch_lora_weights.safetensors lora_weights/


mv: cannot stat 'pytorch_lora_weights.safetensors': No such file or directory


In [23]:


#@title Tiny LoRA after Training - Outputs
from diffusers import StableDiffusionPipeline
import torch
import os
from PIL import Image

# Load base model
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")

# Attach your LoRA weights
pipe.unet.load_attn_procs("lora_weights")

# Define prompts
lora_prompts = [
    "a realistic 3D render of a red and blue chessboard with dramatic lighting",
    "an overhead shot of a wood chessboard in the middle of a game"
]

# Generate & save images
output_dir = "lora_outputs"
os.makedirs(output_dir, exist_ok=True)

for i, p in enumerate(lora_prompts):
    img = pipe(prompt=p, num_inference_steps=30, guidance_scale=8.0).images[0]
    img.save(f"{output_dir}/lora_out_{i}.png")

print("Done. Exported images are in:", os.path.abspath(output_dir))




Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Done. Exported images are in: /content/lora_outputs


In [25]:


!zip -r TrackA.zip lora_outputs outputs_sd

files.download("TrackA.zip")



  adding: lora_outputs/ (stored 0%)
  adding: lora_outputs/lora_out_0.png (deflated 0%)
  adding: lora_outputs/lora_out_1.png (deflated 0%)
  adding: outputs_sd/ (stored 0%)
  adding: outputs_sd/txt2img_1.png (deflated 0%)
  adding: outputs_sd/txt2img_2.png (deflated 0%)
  adding: outputs_sd/txt2img_0.png (deflated 0%)
  adding: outputs_sd/img2img_edit.png (deflated 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>